In [ ]:
import pandas as pd

def analizar_valores_unicos(df, campo):
    """
    Función para analizar los valores únicos en una columna específica de un DataFrame.

    Parámetros:
    - df: DataFrame de Pandas
    - campo: Nombre de la columna a analizar

    Imprime los conteos de valores únicos en el campo especificado.
    """
    print(f"\n--- Analizando valores únicos en el campo '{campo}' ---\n")
    registros = df[campo].value_counts()

    print(f"Conteos de valores únicos en la columna '{campo}':")
    for valor, cuenta in registros.items():
        print(f"{valor} -> {cuenta}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Cargar el archivo CSV en un DataFrame de pandas
input_path_train = '/content/drive/My Drive/0. JOSE IES SV/Entrenamiento_hasta_2017.csv'
df_train_csv = pd.read_csv(input_path_train)

input_path_test= '/content/drive/My Drive/0. JOSE IES SV/Test_2018.csv'
df_test_csv = pd.read_csv(input_path_test)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Encontrar las columnas que tienen un solo valor, no nos sirven de nada para la clasificación

print('---1. Encontrar las columnas que tienen un solo valor en train y test para eliminarlas después ----')
print("")

print(" ------ Columnas con un solo valor en train -------------------")
print("")
print("Columnas con un solo valor en train:")
print("")

columnas_con_un_valor_train = df_train_csv.columns[df_train_csv.nunique() == 1].tolist()
for columna in columnas_con_un_valor_train:
    valor_unico = df_train_csv[columna].unique()[0]
    print(f"Columna: {columna}, Valor único: {valor_unico}")

print("")
print(" ------ Columnas con un solo valor en test  -------------------")
print("")
print("Columnas con un solo valor en test:")
print("")
columnas_con_un_valor_test = df_test_csv.columns[df_test_csv.nunique() == 1].tolist()
for columna in columnas_con_un_valor_test:
    valor_unico = df_test_csv[columna].unique()[0]
    print(f"Columna: {columna}, Valor único: {valor_unico}")
print('---------------------------------------------------------')


---1. Encontrar las columnas que tienen un solo valor en train y test para eliminarlas después ----

 ------ Columnas con un solo valor en train -------------------

Columnas con un solo valor en train:

Columna: procedureAnalysedMedia, Valor único: water

 ------ Columnas con un solo valor en test  -------------------

Columnas con un solo valor en test:

Columna: procedureAnalysedMedia, Valor único: water
---------------------------------------------------------


In [ ]:
# Eliminar columna procedureAnalysedMedia, dado el paso anterior
columnas_eliminar = ['procedureAnalysedMedia']
df_train_csv = df_train_csv.drop(columns=columnas_eliminar)
df_test_csv = df_test_csv.drop(columns=columnas_eliminar)
print(df_train_csv.columns)
print(df_test_csv.columns)

Index(['monitoringSiteIdentifier', 'parameterWaterBodyCategory',
       'observedPropertyDeterminandCode', 'resultUom',
       'phenomenonTimeSamplingDate', 'resultObservedValue',
       'resultQualityObservedValueBelowLOQ', 'procedureLOQValue',
       'procedureAnalyticalMethod', 'parameterSampleDepth',
       'resultObservationStatus', 'metadata_statements'],
      dtype='object')
Index(['monitoringSiteIdentifier', 'parameterWaterBodyCategory',
       'observedPropertyDeterminandCode', 'resultUom',
       'phenomenonTimeSamplingDate', 'resultObservedValue',
       'resultQualityObservedValueBelowLOQ', 'procedureLOQValue',
       'procedureAnalyticalMethod', 'parameterSampleDepth',
       'resultObservationStatus', 'metadata_statements'],
      dtype='object')


In [ ]:
# Filtrar filas donde 'resultObservationStatus' es igual a 'A' (registro confirmado como correcto)
# solo nos interesan los registros válidos
df_train_csv = df_train_csv[df_train_csv['resultObservationStatus'] == 'A']
df_test_csv = df_test_csv[df_test_csv['resultObservationStatus'] == 'A']

# Se filtran filas donde 'procedureLOQValue' <= 'resultObservedValue' , puede así detectar fallos

df_train_csv['procedureLOQValue'].fillna(0, inplace=True)
df_test_csv['procedureLOQValue'].fillna(0, inplace=True)

df_train_csv =  df_train_csv[df_train_csv['procedureLOQValue'] <= df_train_csv['resultObservedValue'] ]
df_test_csv =  df_test_csv[df_test_csv['procedureLOQValue'] <= df_test_csv['resultObservedValue'] ]

# df_train_csv =  df_train_csv[df_train_csv['procedureLOQValue'].notna()]
# df_test_csv =  df_test_csv[df_test_csv['procedureLOQValue'].notna()]

# Encontrar las columnas que tienen valores NaN
columnas_con_nan_train = df_train_csv.loc[:, df_train_csv.isna().any()].columns.tolist()
columnas_con_nan_test = df_test_csv.loc[:, df_test_csv.isna().any()].columns.tolist()
print(columnas_con_nan_train)
print(columnas_con_nan_test)


<ipython-input-53-859c0d97140f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_csv['procedureLOQValue'].fillna(0, inplace=True)
<ipython-input-53-859c0d97140f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_csv['procedureLOQValue'].fillna(0, inplace=True)


['metadata_statements']
['metadata_statements']


In [ ]:
# metadata_statements son metadatos que no tienen importancia en el análisis, se tenía que haber eliminado antes

# Ya filtrado'resultObservationStatus' es igual a 'A' (registro confirmado como correcto)
# con lo que también eliminamos esa columna
columnas_eliminar = ['resultObservationStatus','metadata_statements']
df_train_csv = df_train_csv.drop(columns=columnas_eliminar)
df_test_csv = df_test_csv.drop(columns=columnas_eliminar)

# Encontrar las columnas que siguen teniendo valores NaN
columnas_con_nan_train = df_train_csv.loc[:, df_train_csv.isna().any()].columns.tolist()
columnas_con_nan_test = df_test_csv.loc[:, df_test_csv.isna().any()].columns.tolist()
print("")
print(" --- 1. Columnas que siguen teniendo valores NaN en train ---")
print("")
print(columnas_con_nan_train)
print("")
print(" --- 2. Columnas que siguen teniendo valores NaN en test ---")
print("")
print(columnas_con_nan_test)



 --- 1. Columnas que siguen teniendo valores NaN en train ---

[]

 --- 2. Columnas que siguen teniendo valores NaN en test ---

[]


In [ ]:
# Hay dos valores importantes, resultUom (tipo de unidad de medida) y observedPropertyDeterminandCode (propiedad a medir)
# Analizamos que valores son diferentes en resultUom

analizar_valores_unicos(df_train_csv,'resultUom')
analizar_valores_unicos(df_test_csv,'resultUom')




--- Analizando valores únicos en el campo 'resultUom' ---

Conteos de valores únicos en la columna 'resultUom':
ug/L -> 696434
mg/L -> 115054
mg{P}/L -> 79139
mg{N}/L -> 62730
mg{NH4}/L -> 50577
mg{NO3}/L -> 47966
[pH] -> 41534
uS/cm -> 38398
Cel -> 35449
% -> 34515
mg{NO2}/L -> 32730
mg{O2}/L -> 32351
mg{C}/L -> 31139
m -> 16702
mg{Si}/L -> 13350
mg{NH3}/L -> 9477
mg{CaCO3}/L -> 6442
mmol/L -> 5350
ug{TEQ}/kg -> 2110
mg{S}/L -> 84

--- Analizando valores únicos en el campo 'resultUom' ---

Conteos de valores únicos en la columna 'resultUom':
ug/L -> 598
mg/L -> 27
Cel -> 17
[pH] -> 17
uS/cm -> 16


In [ ]:
# El modelo tiene que aprender del train y del test con estas columnas

print(df_train_csv.columns)
print(df_test_csv.columns)

Index(['monitoringSiteIdentifier', 'parameterWaterBodyCategory',
       'observedPropertyDeterminandCode', 'resultUom',
       'phenomenonTimeSamplingDate', 'resultObservedValue',
       'resultQualityObservedValueBelowLOQ', 'procedureLOQValue',
       'procedureAnalyticalMethod', 'parameterSampleDepth'],
      dtype='object')
Index(['monitoringSiteIdentifier', 'parameterWaterBodyCategory',
       'observedPropertyDeterminandCode', 'resultUom',
       'phenomenonTimeSamplingDate', 'resultObservedValue',
       'resultQualityObservedValueBelowLOQ', 'procedureLOQValue',
       'procedureAnalyticalMethod', 'parameterSampleDepth'],
      dtype='object')


In [ ]:
# Volvemos a encontrar las columnas que tienen un solo valor, no nos sirven de nada para la clasificación
# Lo volvemos a verificar

print('---1. Encontrar las columnas que tienen un solo valor en train y test para eliminarlas después ----')
print("")

print(" ------ Columnas con un solo valor en train -------------------")
print("")
print("Columnas con un solo valor en train:")
print("")

columnas_con_un_valor_train = df_train_csv.columns[df_train_csv.nunique() == 1].tolist()
for columna in columnas_con_un_valor_train:
    valor_unico = df_train_csv[columna].unique()[0]
    print(f"Columna: {columna}, Valor único: {valor_unico}")

print("")
print(" ------ Columnas con un solo valor en test  -------------------")
print("")
print("Columnas con un solo valor en test:")
print("")
columnas_con_un_valor_test = df_test_csv.columns[df_test_csv.nunique() == 1].tolist()
for columna in columnas_con_un_valor_test:
    valor_unico = df_test_csv[columna].unique()[0]
    print(f"Columna: {columna}, Valor único: {valor_unico}")
print('---------------------------------------------------------')




---1. Encontrar las columnas que tienen un solo valor en train y test para eliminarlas después ----

 ------ Columnas con un solo valor en train -------------------

Columnas con un solo valor en train:


 ------ Columnas con un solo valor en test  -------------------

Columnas con un solo valor en test:

---------------------------------------------------------


In [ ]:
# Salvar los archivos dataFrame de pandas en archivos CSV

output_path = '/content/drive/My Drive/0. JOSE IES SV/Entrenamiento_hasta_2017_(simplificado).csv'
df_train_csv.to_csv(output_path,index=False)

output_path = '/content/drive/My Drive/0. JOSE IES SV/Test_2018_(simplificado).csv'
df_test_csv.to_csv(output_path,index=False)
